<a href="https://colab.research.google.com/github/Eman-Nasr/AI-pest-Detection-/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/My Drive/insect"


In [ ]:
import os
print(os.path.exists("/content/drive/My Drive/insect"))


True


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Define optimized image parameters
img_height, img_width = 224, 224  # Higher resolution for better accuracy
batch_size = 32  # Balanced batch size for performance
AUTOTUNE = tf.data.AUTOTUNE

# Define paths for saving model progress
checkpoint_xception = "/content/drive/My Drive/xception_checkpoint.keras"
history_file = "/content/drive/My Drive/xception_history.txt"
final_model_path = "/content/drive/My Drive/insect_classifier_xception.keras"

# Load dataset
new_train_ds_raw = keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/insect",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

new_val_ds = keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/insect",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Get class names BEFORE applying `.map()`
class_names = new_train_ds_raw.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Apply Advanced Data Augmentation
new_data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
])

# Apply `.map()` AFTER getting class names
new_train_ds = new_train_ds_raw.map(lambda x, y: (new_data_augmentation(x, training=True), y))
new_train_ds = new_train_ds.cache().prefetch(buffer_size=AUTOTUNE)  # Speed up training
new_val_ds = new_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained Xception Model (Higher Accuracy than MobileNetV2)
xception_base = keras.applications.Xception(input_shape=(img_height, img_width, 3),
                                            include_top=False,
                                            weights='imagenet')

# Unfreeze last 30 layers for fine-tuning
for layer in xception_base.layers[-30:]:
    layer.trainable = True

# Build the Fine-Tuned Model
xception_model = keras.Sequential([
    xception_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),  # Reduced dropout for better learning
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

# Use an optimized learning rate with decay
xception_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Save Training Progress (Checkpoint)
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_xception,
                                                      save_best_only=True,
                                                      monitor='val_accuracy',
                                                      mode='max')

# Learning Rate Adjustment
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Define the number of additional epochs to train
additional_epochs = 3
last_epoch = 0  # Default to starting from scratch

# **Step 1: Check for a saved history file**
if os.path.exists(history_file):
    with open(history_file, "r") as f:
        content = f.read().strip()  # Read and strip whitespace
        if content.isdigit():
            last_epoch = int(content)  # Convert only if valid number
            print(f"Resuming training from epoch {last_epoch}...")
        else:
            print("History file is empty. Starting training from scratch.")
            last_epoch = 0  # Start from scratch
else:
    print("History file not found. Training will start from scratch.")

# **Step 2: Load model checkpoint if available**
if os.path.exists(checkpoint_xception):
    xception_model = keras.models.load_model(checkpoint_xception)
    print("Loaded checkpoint model from:", checkpoint_xception)

# **Step 3: Train the model from the last saved epoch**
history_xception = xception_model.fit(new_train_ds, validation_data=new_val_ds,
                                      epochs=last_epoch + additional_epochs,
                                      initial_epoch=last_epoch,
                                      callbacks=[checkpoint_callback, lr_scheduler])

# **Step 4: Save the last trained epoch number**
with open(history_file, "w") as f:
    f.write(str(last_epoch + additional_epochs))

# **Step 5: Save final model after training**
xception_model.save(final_model_path)
print("Model training continued and saved successfully.")

Found 6772 files belonging to 2 classes.
Using 5418 files for training.
Found 6772 files belonging to 2 classes.
Using 1354 files for validation.
Classes: ['harmful', 'not_harm']
History file not found. Training will start from scratch.
Loaded checkpoint model from: /content/drive/My Drive/xception_checkpoint.keras
Epoch 1/3
170/170 ━━━━━━━━━━━━━━━━━━━━ 5840s 34s/step - accuracy: 0.9278 - loss: 0.1969 - val_accuracy: 0.4963 - val_loss: 1.9849 - learning_rate: 3.0000e-04
Epoch 2/3
170/170 ━━━━━━━━━━━━━━━━━━━━ 5700s 33s/step - accuracy: 0.9815 - loss: 0.0625 - val_accuracy: 0.7740 - val_loss: 1.6950 - learning_rate: 3.0000e-04
Epoch 3/3
170/170 ━━━━━━━━━━━━━━━━━━━━ 5730s 33s/step - accuracy: 0.9904 - loss: 0.0270 - val_accuracy: 0.6928 - val_loss: 1.3905 - learning_rate: 3.0000e-04
Model training continued and saved successfully.


In [ ]:
!ls "/content/drive/My Drive/"


'Colab Notebooks'		     mobilenet_checkpoint.keras
 Data				     mobilenet_training_history.json
 harmful_insect_classifier.keras     not_harm_insect_classifier.keras
 insect				     risk_classifier.keras
 insect_classifier.keras	     xception_checkpoint.keras
 insect_classifier_mobilenet.keras   xception_history.json
 insect_classifier_xception.keras    xception_history.txt
 insect_training_history.json


In [ ]:
!pip install tensorflow


In [2]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plots training and validation accuracy and loss.

    Parameters:
        history: Keras history object returned from model.fit()
    """
    epochs = range(1, len(history.history["loss"]) + 1)

    # Plot Accuracy
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, history.history["accuracy"], label="Training Accuracy")
    plt.plot(epochs, history.history["val_accuracy"], label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()
    plt.show()

    # Plot Loss
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, history.history["loss"], label="Training Loss")
    plt.plot(epochs, history.history["val_loss"], label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()